In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from PIL import Image

import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.nn import functional as F
from torch import optim

from torch.utils.data import DataLoader,Dataset, random_split

from sklearn.model_selection import train_test_split


In [2]:
TRAIN_DATA = '../data/train.csv'
TEST_DATA = '../data/test.csv'
TRAIN_IMG = '../data/train_test_data/train/'

In [3]:
train_df = pd.read_csv(TRAIN_DATA)
train_df.head()

,label,latitude,longitude,year,example_path
0,0,-2.051853,111.826093,2001,train_test_data/train/1297.png
1,2,-1.989349,105.309496,2013,train_test_data/train/1199.png
2,0,1.223256,100.702217,2014,train_test_data/train/1348.png
3,0,-2.342948,103.890226,2008,train_test_data/train/2214.png
4,0,-0.126555,101.758175,2011,train_test_data/train/2220.png


In [4]:
test_df = pd.read_csv(TEST_DATA)
test_df.head()

,latitude,longitude,year,example_path
0,0.761681,122.755954,2006,train_test_data/test/69.png
1,-8.059785,113.053791,2007,train_test_data/test/469.png
2,-2.006610,111.746316,2002,train_test_data/test/6.png
3,0.901765,114.042495,2016,train_test_data/test/351.png
4,1.911210,100.829633,2008,train_test_data/test/1001.png


In [5]:
img = Image.open(TRAIN_IMG+'2220.png')
img_array = np.asarray(img)
px.imshow(img_array)

In [6]:
img_array[0][0]

array([35, 31, 14], dtype=uint8)

In [7]:
img.size

(332, 332)

In [8]:
img.mode

'RGB'

In [9]:
def imshow(img):
  img = img / 2 + 0.5 
  npimg = img.numpy() 
  px.imshow(np.transpose(npimg, (1, 2, 0)))
  

# Custom Dataset and Dataloader

In [31]:
class ImageDataset(Dataset):
  def __init__(self,df,data_folder,transform):
    self.df = df
    self.transform = transform
    self.img_folder = data_folder
     
    self.image_names = self.df[:]['example_path']
    self.labels = self.df[:]['label']
   
  def __len__(self):
    return len(self.image_names)
 
  def __getitem__(self,index):
     
    image=Image.open(self.img_folder+self.image_names.iloc[index])

    image=self.transform(image)
    targets=self.labels[index]
     
    sample = {'image': image,'labels':targets}
 
    return sample

train_transform = transforms.Compose([
                       transforms.RandomHorizontalFlip(p=1),
                       transforms.RandomVerticalFlip(p=1),
                       transforms.RandomRotation(degrees=0.66),
                       transforms.ToTensor(), 
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


data = ImageDataset(train_df, '../data/', train_transform)
DL = DataLoader(data, batch_size=20,shuffle=True)
sample = next(iter(DL))
print(sample['labels'][0])
img = (sample['image'][0] / 2 + 0.5 )
npimg = img.numpy() 
px.imshow(np.transpose(npimg, (1, 2, 0)))

tensor(2)


## Split Train test

In [32]:
train_data, test_data = random_split(data, [0.8, 0.2])
print(f'DATA: {len(data)}, TRAIN: {len(train_data)}, TEST: {len(test_data)}')

DATA: 1714, TRAIN: 1372, TEST: 342


In [33]:
train_DL = DataLoader(train_data, batch_size=20, shuffle=True)


## DEFINING PARAMETERS

In [34]:
from model import Net

net = Net(list(sample['image'][0].shape), classes=3)
net

Net(
  (conv): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=102400, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=3, bias=True)
  )
)

In [36]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())


# TRAINING

In [37]:
import time
start = time.time()




for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_DL, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data['image']
        labels = data['labels']

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

# whatever you are timing goes here
total_time = time.time() - start

# Waits for everything to finish running
#
#torch.cuda.synchronize()

print('Finished Training')
print(total_time/60)   

[1,    10] loss: 1.263
[1,    20] loss: 0.964
[1,    30] loss: 0.983
[1,    40] loss: 1.014
[1,    50] loss: 0.986
[1,    60] loss: 1.009
[2,    10] loss: 0.985
[2,    20] loss: 0.962
[2,    30] loss: 0.961
[2,    40] loss: 0.966
[2,    50] loss: 0.954
[2,    60] loss: 0.957
[3,    10] loss: 0.973
[3,    20] loss: 0.962
[3,    30] loss: 0.985
[3,    40] loss: 0.976
[3,    50] loss: 0.896
[3,    60] loss: 0.990
[4,    10] loss: 0.952
[4,    20] loss: 0.990
[4,    30] loss: 0.952
[4,    40] loss: 1.002
[4,    50] loss: 0.944
[4,    60] loss: 1.003
[5,    10] loss: 0.922
[5,    20] loss: 0.985
[5,    30] loss: 0.979
[5,    40] loss: 0.931
[5,    50] loss: 0.928
[5,    60] loss: 1.287
[6,    10] loss: 0.961
[6,    20] loss: 0.882
[6,    30] loss: 0.963
[6,    40] loss: 0.969
[6,    50] loss: 0.984
[6,    60] loss: 0.969
[7,    10] loss: 0.970
[7,    20] loss: 0.907
[7,    30] loss: 0.886
[7,    40] loss: 0.839
[7,    50] loss: 0.913
[7,    60] loss: 0.884
[8,    10] loss: 0.807
[8,    20] 

In [16]:
print(total_time/60)  # milliseconds



1.3456761360168457


## CALCULATE SCORE

In [38]:
test_DL = DataLoader(test_data, batch_size=20, shuffle=True)

test_sample = next(iter(test_DL))
test_sample

{'image': tensor([[[[-1.0000, -1.0000, -1.0000,  ..., -0.8588, -0.8667, -1.0000],
           [-0.9529, -0.9608, -0.9451,  ..., -0.8353, -0.8588, -1.0000],
           [-0.9608, -0.9373, -0.9294,  ..., -0.7569, -0.7804, -1.0000],
           ...,
           [-1.0000, -0.9059, -0.9216,  ..., -0.7804, -0.8275, -0.8588],
           [-1.0000, -0.9059, -0.9059,  ..., -0.8118, -0.8431, -0.8510],
           [-1.0000, -0.9059, -0.9137,  ..., -1.0000, -1.0000, -1.0000]],
 
          [[-1.0000, -1.0000, -1.0000,  ..., -0.8353, -0.8431, -1.0000],
           [-0.9294, -0.9294, -0.9137,  ..., -0.8353, -0.8588, -1.0000],
           [-0.9373, -0.9137, -0.9059,  ..., -0.7725, -0.7961, -1.0000],
           ...,
           [-1.0000, -0.8667, -0.8667,  ..., -0.7804, -0.8275, -0.8510],
           [-1.0000, -0.8745, -0.8667,  ..., -0.8196, -0.8275, -0.8353],
           [-1.0000, -0.8745, -0.8824,  ..., -1.0000, -1.0000, -1.0000]],
 
          [[-1.0000, -1.0000, -1.0000,  ..., -0.9843, -0.9843, -1.0000],
    

In [39]:



y_true, y_pred = net.predict(test_DL)

len(y_true)


342

In [43]:
y_pred.size()

torch.Size([342, 1])

In [40]:
from torchmetrics import F1Score

F1 = F1Score(num_classes=3, average='macro')
score = F1(y_pred.flatten(), y_true)

In [41]:
score

tensor(0.3544)

In [20]:
test_DL

In [21]:
test_data.indices

[546,
 107,
 111,
 1522,
 539,
 224,
 803,
 498,
 373,
 557,
 1405,
 1375,
 418,
 213,
 163,
 64,
 1234,
 183,
 1364,
 32,
 30,
 1401,
 472,
 1046,
 1583,
 1185,
 165,
 540,
 1639,
 687,
 1150,
 1538,
 885,
 1054,
 1515,
 1257,
 1151,
 898,
 590,
 250,
 1660,
 1324,
 1687,
 825,
 859,
 1044,
 1662,
 1237,
 476,
 1356,
 556,
 1261,
 1208,
 616,
 636,
 763,
 206,
 171,
 126,
 1483,
 966,
 799,
 884,
 1205,
 592,
 874,
 929,
 956,
 76,
 1413,
 1479,
 987,
 772,
 1436,
 6,
 2,
 154,
 1494,
 193,
 1283,
 754,
 1478,
 1460,
 838,
 752,
 36,
 80,
 179,
 267,
 1105,
 317,
 928,
 1461,
 1057,
 1049,
 1123,
 1599,
 29,
 1301,
 68,
 500,
 1080,
 875,
 702,
 793,
 465,
 1231,
 1407,
 917,
 78,
 610,
 944,
 783,
 896,
 1317,
 356,
 285,
 984,
 814,
 331,
 1681,
 717,
 1300,
 1310,
 293,
 1041,
 1621,
 17,
 1423,
 635,
 194,
 422,
 805,
 1591,
 786,
 1651,
 1403,
 487,
 1476,
 629,
 599,
 1168,
 278,
 1437,
 168,
 1032,
 1710,
 1453,
 1220,
 86,
 341,
 1613,
 952,
 1511,
 1305,
 429,
 127,
 708,
 13